In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [4]:
from sklearn.preprocessing import *
from sklearn.model_selection import *

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

In [6]:
df = pd.read_parquet('./parquet/preprocessed.parquet')

Drop unnecessary columns


In [7]:
df = df.drop(['ID', 'Dt_Customer', 'Kidhome', 'Teenhome'],
             axis=1).reset_index(drop=True)

In [8]:
columns = df.columns
columns_to_drop = [col for col in columns if col.startswith('Marital_')]
df = df.drop(columns=columns_to_drop)

## Modelling


In [9]:
from sklearn.svm import *
from sklearn.naive_bayes import *
from sklearn.tree import *
from sklearn.neighbors import *
from sklearn.linear_model import *

In [10]:
X = df.drop('Response', axis=1)
y = df.Response

pd.set_option('display.max_columns', 50)
X

,Year_Birth,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Total_Children,Days_Since_Customer,Education_Basic,Education_Graduation,Education_Master,Education_PhD,A_Marital_Status_Married_Kidhome,A_Marital_Status_Married_Teenhome,A_Marital_Status_Single_Kidhome,A_Marital_Status_Single_Teenhome,A_Marital_Status_Together_Kidhome,A_Marital_Status_Together_Teenhome,A_Marital_Status_Widow_Kidhome,A_Marital_Status_Widow_Teenhome
0,1961,57091.0,0,464,5,64,7,0,37,1,7,3,7,5,0,0,3544,False,True,False,False,0,0,0,0,0,0,0,0
1,1958,67267.0,0,134,11,59,15,2,30,1,3,2,5,2,0,1,3577,False,True,False,False,0,1,0,0,0,0,0,0
2,1967,32474.0,0,10,0,1,0,0,0,1,1,0,2,7,0,2,3401,False,True,False,False,0,0,0,0,1,1,0,0
3,1989,21474.0,0,6,16,24,11,0,34,2,3,1,2,7,0,1,3494,False,True,False,False,0,0,1,0,0,0,0,0
4,1967,44931.0,0,78,0,11,0,0,7,1,2,1,3,5,0,1,3692,False,True,False,False,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1209,1974,20130.0,99,0,6,3,7,6,12,1,1,0,3,8,0,0,3634,False,False,False,False,0,0,0,0,0,0,0,0
1210,1973,42429.0,99,55,0,6,2,0,4,2,1,1,3,5,0,1,3404,False,True,False,False,0,0,0,0,0,0,0,1
1211,1973,42429.0,99,55,0,6,2,0,4,2,1,1,3,5,0,1,3404,False,True,False,False,0,0,0,0,0,0,0,1
1212,1977,31056.0,99,5,10,13,3,8,16,1,1,0,3,8,0,1,4053,False,False,False,False,1,0,0,0,0,0,0,0


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=12)

In [12]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
pf = PolynomialFeatures()
X_train = pf.fit_transform(X_train)
X_test = pf.transform(X_test)

In [14]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)

In [15]:
def assess_cv(model, name):
    accuracy = cross_val_score(
        model, X_train, y_train, cv=skf, scoring='accuracy')
    precision = cross_val_score(
        model, X_train, y_train, cv=skf, scoring='precision')
    recall = cross_val_score(model, X_train, y_train, cv=skf, scoring='recall')
    f1 = cross_val_score(model, X_train, y_train, cv=skf, scoring='f1')
    roc = cross_val_score(model, X_train, y_train, cv=skf, scoring='roc_auc')

    eval = {name: {} for n in range(5)}
    eval[name]['Accuracy'] = accuracy.mean()
    eval[name]['Precision'] = precision.mean()
    eval[name]['Recall'] = recall.mean()
    eval[name]['F1 Score'] = f1.mean()
    eval[name]['ROC AUC'] = roc.mean()
    return pd.DataFrame(eval).T

In [16]:
overview = pd.DataFrame({
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1 Score': [],
    'ROC AUC': []
})

### Logistic Regression


In [17]:
params = {
    'max_iter': [50, 70, 100, 200, 500, 1000, 2000, 5000]
}

In [18]:
clf_lr = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=params,
    scoring='f1',
    cv=skf
)

In [19]:
clf_lr.fit(X_train, y_train)

C:\Users\araza\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\araza\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=12, shuffle=True),
             estimator=LogisticRegression(),
             param_grid={'max_iter': [50, 70, 100, 200, 500, 1000, 2000, 5000]},
             scoring='f1')

In [20]:
clf_lr.best_estimator_

LogisticRegression()

In [21]:
pd.DataFrame(clf_lr.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.029814,0.004411,0.002431,0.000438,50,{'max_iter': 50},0.476190,0.378378,0.375000,0.370370,0.4,0.399988,0.039434,8
1,0.037240,0.004941,0.002130,0.000544,70,{'max_iter': 70},0.465116,0.388889,0.363636,0.384615,0.4,0.400451,0.034414,7
2,0.043588,0.004302,0.002400,0.000492,100,{'max_iter': 100},0.439024,0.432432,0.411765,0.384615,0.4,0.413567,0.020147,1
3,0.046156,0.002622,0.002605,0.000373,200,{'max_iter': 200},0.439024,0.432432,0.411765,0.384615,0.4,0.413567,0.020147,1
4,0.055378,0.011418,0.002647,0.000580,500,{'max_iter': 500},0.439024,0.432432,0.411765,0.384615,0.4,0.413567,0.020147,1
5,0.048683,0.002793,0.002503,0.000473,1000,{'max_iter': 1000},0.439024,0.432432,0.411765,0.384615,0.4,0.413567,0.020147,1
6,0.056376,0.004526,0.002700,0.000749,2000,{'max_iter': 2000},0.439024,0.432432,0.411765,0.384615,0.4,0.413567,0.020147,1
7,0.059017,0.006692,0.003322,0.001108,5000,{'max_iter': 5000},0.439024,0.432432,0.411765,0.384615,0.4,0.413567,0.020147,1


In [22]:
print(pd.DataFrame(clf_lr.cv_results_)[['param_max_iter', 'mean_test_score', 'rank_test_score']].to_latex(index=False))

\begin{tabular}{lrr}
\toprule
param_max_iter & mean_test_score & rank_test_score \\
\midrule
50 & 0.399988 & 8 \\
70 & 0.400451 & 7 \\
100 & 0.413567 & 1 \\
200 & 0.413567 & 1 \\
500 & 0.413567 & 1 \\
1000 & 0.413567 & 1 \\
2000 & 0.413567 & 1 \\
5000 & 0.413567 & 1 \\
\bottomrule
\end{tabular}



In [23]:
# scores = cross_val_score(clf_lr.best_estimator_, X_train, y_train, cv=skf, scoring='roc_auc')
assess_cv(clf_lr.best_estimator_, 'Logistic Regression')

,Accuracy,F1 Score,Precision,ROC AUC,Recall
Logistic Regression,0.899091,0.413567,0.474971,0.771246,0.388889


In [24]:
overview = pd.concat([overview, assess_cv(
    clf_lr.best_estimator_, 'Logistic Regression')])

### SVM


In [25]:
params = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

In [26]:
clf_svc = GridSearchCV(
    estimator=SVC(),
    param_grid=params,
    scoring='f1',
    cv=skf
)

In [27]:
clf_svc.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=12, shuffle=True),
             estimator=SVC(),
             param_grid={'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='f1')

In [28]:
clf_svc.best_estimator_

SVC(kernel='linear')

In [29]:
pd.DataFrame(clf_svc.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.034972,0.002648,0.006120,0.000672,scale,linear,"{'gamma': 'scale', 'kernel': 'linear'}",0.409091,0.363636,0.473684,0.307692,0.411765,0.393174,0.055246,1
1,0.035581,0.002710,0.008953,0.000392,scale,poly,"{'gamma': 'scale', 'kernel': 'poly'}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4
2,0.032733,0.002179,0.018088,0.000829,scale,rbf,"{'gamma': 'scale', 'kernel': 'rbf'}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4
3,0.023378,0.001233,0.006715,0.000397,scale,sigmoid,"{'gamma': 'scale', 'kernel': 'sigmoid'}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4
4,0.031274,0.002991,0.005808,0.000614,auto,linear,"{'gamma': 'auto', 'kernel': 'linear'}",0.409091,0.363636,0.473684,0.307692,0.411765,0.393174,0.055246,1
5,0.031259,0.001017,0.008720,0.000379,auto,poly,"{'gamma': 'auto', 'kernel': 'poly'}",0.153846,0.000000,0.083333,0.000000,0.166667,0.080769,0.071795,3
6,0.036399,0.002097,0.022565,0.000550,auto,rbf,"{'gamma': 'auto', 'kernel': 'rbf'}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4
7,0.019894,0.000564,0.006112,0.001081,auto,sigmoid,"{'gamma': 'auto', 'kernel': 'sigmoid'}",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4


In [30]:
print(pd.DataFrame(clf_svc.cv_results_)[['param_gamma', 'param_kernel', 'mean_test_score', 'rank_test_score']].to_latex(index=False))

\begin{tabular}{llrr}
\toprule
param_gamma & param_kernel & mean_test_score & rank_test_score \\
\midrule
scale & linear & 0.393174 & 1 \\
scale & poly & 0.000000 & 4 \\
scale & rbf & 0.000000 & 4 \\
scale & sigmoid & 0.000000 & 4 \\
auto & linear & 0.393174 & 1 \\
auto & poly & 0.080769 & 3 \\
auto & rbf & 0.000000 & 4 \\
auto & sigmoid & 0.000000 & 4 \\
\bottomrule
\end{tabular}



In [31]:
# scores = cross_val_score(clf_svc.best_estimator_, X_train, y_train, cv=skf, scoring='precision')
assess_cv(clf_svc.best_estimator_, 'SVM')

,Accuracy,F1 Score,Precision,ROC AUC,Recall
SVM,0.875395,0.393174,0.365412,0.752352,0.433333


In [32]:
overview = pd.concat([overview, assess_cv(clf_svc.best_estimator_, 'SVM')])

### Naive Bayes


In [33]:
params = {
    'alpha': [1.0, 2.0, 5.0, 7, 10],
    'fit_prior': [True, False],
}

In [34]:
clf_nb = GridSearchCV(
    estimator=BernoulliNB(),
    param_grid=params,
    scoring='f1',
    cv=skf
)

In [35]:
clf_nb.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=12, shuffle=True),
             estimator=BernoulliNB(),
             param_grid={'alpha': [1.0, 2.0, 5.0, 7, 10],
                         'fit_prior': [True, False]},
             scoring='f1')

In [36]:
clf_nb.best_estimator_

BernoulliNB(alpha=7)

In [37]:
pd.DataFrame(clf_nb.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010739,0.003211,0.004227,0.000230,1.0,True,"{'alpha': 1.0, 'fit_prior': True}",0.318841,0.339623,0.387097,0.310345,0.253521,0.321885,0.043320,7
1,0.011143,0.001495,0.003450,0.000865,1.0,False,"{'alpha': 1.0, 'fit_prior': False}",0.301370,0.379310,0.346667,0.303030,0.337662,0.333608,0.029155,4
2,0.009075,0.002668,0.003610,0.000799,2.0,True,"{'alpha': 2.0, 'fit_prior': True}",0.318841,0.333333,0.393443,0.310345,0.277778,0.326748,0.038003,5
3,0.009218,0.000944,0.003886,0.000604,2.0,False,"{'alpha': 2.0, 'fit_prior': False}",0.301370,0.372881,0.346667,0.272727,0.325000,0.323729,0.034770,6
4,0.008461,0.000464,0.003518,0.000641,5.0,True,"{'alpha': 5.0, 'fit_prior': True}",0.303030,0.339623,0.393443,0.315789,0.338028,0.337983,0.030960,3
5,0.014652,0.003252,0.005765,0.001536,5.0,False,"{'alpha': 5.0, 'fit_prior': False}",0.301370,0.338983,0.320000,0.303030,0.329412,0.318559,0.014653,9
6,0.009544,0.001215,0.003465,0.001034,7,True,"{'alpha': 7, 'fit_prior': True}",0.298507,0.367347,0.406780,0.315789,0.338028,0.345290,0.038416,1
7,0.007824,0.001213,0.003533,0.000607,7,False,"{'alpha': 7, 'fit_prior': False}",0.281690,0.305085,0.320000,0.312500,0.305882,0.305031,0.012848,10
8,0.007299,0.000671,0.003018,0.000447,10,True,"{'alpha': 10, 'fit_prior': True}",0.285714,0.367347,0.350877,0.344828,0.347826,0.339318,0.027912,2
9,0.007451,0.000752,0.003271,0.000260,10,False,"{'alpha': 10, 'fit_prior': False}",0.285714,0.327273,0.342857,0.317460,0.325000,0.319661,0.018877,8


In [38]:
print(pd.DataFrame(clf_nb.cv_results_)[['param_alpha', 'param_fit_prior', 'mean_test_score', 'rank_test_score']].to_latex(index=False))

\begin{tabular}{llrr}
\toprule
param_alpha & param_fit_prior & mean_test_score & rank_test_score \\
\midrule
1.000000 & True & 0.321885 & 7 \\
1.000000 & False & 0.333608 & 4 \\
2.000000 & True & 0.326748 & 5 \\
2.000000 & False & 0.323729 & 6 \\
5.000000 & True & 0.337983 & 3 \\
5.000000 & False & 0.318559 & 9 \\
7 & True & 0.345290 & 1 \\
7 & False & 0.305031 & 10 \\
10 & True & 0.339318 & 2 \\
10 & False & 0.319661 & 8 \\
\bottomrule
\end{tabular}



In [39]:
# scores = cross_val_score(clf_nb, X_train, y_train, cv=skf, scoring='f1')
assess_cv(clf_nb.best_estimator_, 'Naive Bayes')

,Accuracy,F1 Score,Precision,ROC AUC,Recall
Naive Bayes,0.795094,0.34529,0.248854,0.787395,0.577778


In [40]:
tt = pd.concat([assess_cv(clf_nb.best_estimator_, 'Bernoulli NB'), assess_cv(GaussianNB(), 'Gaussian NB')])

In [41]:
print(tt.to_latex())

\begin{tabular}{lrrrrr}
\toprule
 & Accuracy & F1 Score & Precision & ROC AUC & Recall \\
\midrule
Bernoulli NB & 0.795094 & 0.345290 & 0.248854 & 0.787395 & 0.577778 \\
Gaussian NB & 0.294560 & 0.171189 & 0.096033 & 0.536592 & 0.788889 \\
\bottomrule
\end{tabular}



In [42]:
tt

,Accuracy,F1 Score,Precision,ROC AUC,Recall
Bernoulli NB,0.795094,0.345290,0.248854,0.787395,0.577778
Gaussian NB,0.294560,0.171189,0.096033,0.536592,0.788889


In [33]:
overview = pd.concat([overview, assess_cv(clf_nb, 'Naive Bayes')])

### Decision Tree


In [43]:
params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'min_samples_split': [2, 5, 10]
}

In [44]:
clf_dt = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=params,
    scoring='f1',
    cv=skf
)

In [45]:

clf_dt.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=12, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             scoring='f1')

In [46]:
clf_dt.best_estimator_

DecisionTreeClassifier(criterion='log_loss')

In [47]:
pd.DataFrame(clf_dt.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_min_samples_split,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.150777,0.026160,0.002432,0.000384,gini,2,best,"{'criterion': 'gini', 'min_samples_split': 2, ...",0.368421,0.450000,0.428571,0.266667,0.344828,0.371697,0.065008,11
1,0.023361,0.003484,0.002605,0.000375,gini,2,random,"{'criterion': 'gini', 'min_samples_split': 2, ...",0.410256,0.368421,0.315789,0.296296,0.342857,0.346724,0.040055,14
2,0.153891,0.026300,0.002310,0.000406,gini,5,best,"{'criterion': 'gini', 'min_samples_split': 5, ...",0.378378,0.358974,0.292683,0.193548,0.322581,0.309233,0.064947,18
3,0.021171,0.003067,0.002119,0.000202,gini,5,random,"{'criterion': 'gini', 'min_samples_split': 5, ...",0.277778,0.500000,0.307692,0.342857,0.439024,0.373470,0.083340,9
4,0.154163,0.028991,0.002111,0.000366,gini,10,best,"{'criterion': 'gini', 'min_samples_split': 10,...",0.352941,0.350000,0.300000,0.357143,0.285714,0.329160,0.030069,15
5,0.020624,0.001782,0.002111,0.000223,gini,10,random,"{'criterion': 'gini', 'min_samples_split': 10,...",0.400000,0.375000,0.391304,0.357143,0.368421,0.378374,0.015467,8
6,0.097165,0.005071,0.002192,0.000420,entropy,2,best,"{'criterion': 'entropy', 'min_samples_split': ...",0.292683,0.486486,0.325581,0.529412,0.294118,0.385656,0.101454,7
7,0.017160,0.000947,0.002399,0.000490,entropy,2,random,"{'criterion': 'entropy', 'min_samples_split': ...",0.388889,0.421053,0.390244,0.205128,0.457143,0.372491,0.087314,10
8,0.102041,0.006399,0.002488,0.000462,entropy,5,best,"{'criterion': 'entropy', 'min_samples_split': ...",0.315789,0.500000,0.350000,0.611111,0.303030,0.415986,0.120237,2
9,0.016672,0.000994,0.001732,0.000519,entropy,5,random,"{'criterion': 'entropy', 'min_samples_split': ...",0.388889,0.210526,0.425532,0.294118,0.270270,0.317867,0.078724,17


In [52]:
print(pd.DataFrame(clf_dt.cv_results_)[['param_criterion', 'param_min_samples_split', 'param_splitter', 'mean_test_score', 'rank_test_score']].to_latex(index=False))

\begin{tabular}{lllrr}
\toprule
param_criterion & param_min_samples_split & param_splitter & mean_test_score & rank_test_score \\
\midrule
gini & 2 & best & 0.371697 & 11 \\
gini & 2 & random & 0.346724 & 14 \\
gini & 5 & best & 0.309233 & 18 \\
gini & 5 & random & 0.373470 & 9 \\
gini & 10 & best & 0.329160 & 15 \\
gini & 10 & random & 0.378374 & 8 \\
entropy & 2 & best & 0.385656 & 7 \\
entropy & 2 & random & 0.372491 & 10 \\
entropy & 5 & best & 0.415986 & 2 \\
entropy & 5 & random & 0.317867 & 17 \\
entropy & 10 & best & 0.391090 & 4 \\
entropy & 10 & random & 0.361705 & 13 \\
log_loss & 2 & best & 0.416715 & 1 \\
log_loss & 2 & random & 0.388316 & 6 \\
log_loss & 5 & best & 0.390513 & 5 \\
log_loss & 5 & random & 0.365806 & 12 \\
log_loss & 10 & best & 0.399050 & 3 \\
log_loss & 10 & random & 0.324862 & 16 \\
\bottomrule
\end{tabular}



In [51]:
assess_cv(clf_dt.best_estimator_, 'Decision Tree')

,Accuracy,F1 Score,Precision,ROC AUC,Recall
Decision Tree,0.88673,0.396825,0.422027,0.684868,0.433333


In [40]:
overview = pd.concat([overview, assess_cv(
    clf_dt.best_estimator_, 'Decision Tree')])

### K-Nearest Neighbors


In [54]:
params = {
    'n_neighbors': [1, 2, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
}

In [55]:
clf_kn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=params,
    scoring='f1',
    cv=skf
)

In [56]:
clf_kn.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=12, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 2, 5, 7, 10],
                         'weights': ['uniform', 'distance']},
             scoring='f1')

In [57]:
clf_kn.best_estimator_

KNeighborsClassifier(n_neighbors=2, weights='distance')

In [65]:
pd.DataFrame(clf_kn.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,0.002310,0.000395,0.008905,0.000734,auto,2,distance,"{'algorithm': 'auto', 'n_neighbors': 2, 'weigh...",0.277778,0.352941,0.187500,0.296296,0.375000,0.297903,0.065680,1
33,0.002199,0.000253,0.009276,0.000470,brute,2,distance,"{'algorithm': 'brute', 'n_neighbors': 2, 'weig...",0.277778,0.352941,0.187500,0.296296,0.375000,0.297903,0.065680,1
13,0.012905,0.001164,0.074550,0.001938,ball_tree,2,distance,"{'algorithm': 'ball_tree', 'n_neighbors': 2, '...",0.277778,0.352941,0.187500,0.296296,0.375000,0.297903,0.065680,1
23,0.018506,0.001011,0.090718,0.002407,kd_tree,2,distance,"{'algorithm': 'kd_tree', 'n_neighbors': 2, 'we...",0.277778,0.352941,0.187500,0.296296,0.375000,0.297903,0.065680,1
0,0.002908,0.000940,0.048356,0.079417,auto,1,uniform,"{'algorithm': 'auto', 'n_neighbors': 1, 'weigh...",0.277778,0.352941,0.181818,0.296296,0.375000,0.296767,0.067602,5
1,0.002101,0.000668,0.008907,0.000618,auto,1,distance,"{'algorithm': 'auto', 'n_neighbors': 1, 'weigh...",0.277778,0.352941,0.181818,0.296296,0.375000,0.296767,0.067602,5
31,0.002306,0.000510,0.008934,0.000368,brute,1,distance,"{'algorithm': 'brute', 'n_neighbors': 1, 'weig...",0.277778,0.352941,0.181818,0.296296,0.375000,0.296767,0.067602,5
30,0.002501,0.000648,0.008907,0.000504,brute,1,uniform,"{'algorithm': 'brute', 'n_neighbors': 1, 'weig...",0.277778,0.352941,0.181818,0.296296,0.375000,0.296767,0.067602,5
11,0.013148,0.000967,0.078405,0.011031,ball_tree,1,distance,"{'algorithm': 'ball_tree', 'n_neighbors': 1, '...",0.277778,0.352941,0.181818,0.296296,0.375000,0.296767,0.067602,5
10,0.013463,0.001023,0.084553,0.002938,ball_tree,1,uniform,"{'algorithm': 'ball_tree', 'n_neighbors': 1, '...",0.277778,0.352941,0.181818,0.296296,0.375000,0.296767,0.067602,5


In [67]:
print(pd.DataFrame(clf_kn.cv_results_).sort_values(by='rank_test_score')[['param_algorithm', 'param_n_neighbors', 'param_weights', 'mean_test_score', 'rank_test_score']].to_latex(index=False))

\begin{tabular}{lllrr}
\toprule
param_algorithm & param_n_neighbors & param_weights & mean_test_score & rank_test_score \\
\midrule
auto & 2 & distance & 0.297903 & 1 \\
brute & 2 & distance & 0.297903 & 1 \\
ball_tree & 2 & distance & 0.297903 & 1 \\
kd_tree & 2 & distance & 0.297903 & 1 \\
auto & 1 & uniform & 0.296767 & 5 \\
auto & 1 & distance & 0.296767 & 5 \\
brute & 1 & distance & 0.296767 & 5 \\
brute & 1 & uniform & 0.296767 & 5 \\
ball_tree & 1 & distance & 0.296767 & 5 \\
ball_tree & 1 & uniform & 0.296767 & 5 \\
kd_tree & 1 & distance & 0.295630 & 11 \\
kd_tree & 1 & uniform & 0.295630 & 11 \\
ball_tree & 5 & distance & 0.236236 & 13 \\
auto & 5 & distance & 0.236236 & 13 \\
brute & 5 & distance & 0.236236 & 13 \\
kd_tree & 5 & distance & 0.236236 & 13 \\
kd_tree & 10 & distance & 0.197719 & 17 \\
ball_tree & 10 & distance & 0.197719 & 17 \\
brute & 10 & distance & 0.197719 & 17 \\
auto & 10 & distance & 0.197719 & 17 \\
brute & 7 & distance & 0.181053 & 21 \\
kd_tree & 7 &

In [46]:
assess_cv(clf_kn.best_estimator_, 'K-Nearest Neighbors')

,Accuracy,F1 Score,Precision,ROC AUC,Recall
K-Nearest Neighbors,0.883643,0.297903,0.348016,0.6289,0.266667


In [47]:
overview = pd.concat([overview, assess_cv(
    clf_kn.best_estimator_, 'K-Nearest Neighbors')])

In [48]:
overview

,Accuracy,Precision,Recall,F1 Score,ROC AUC
Logistic Regression,0.899091,0.474971,0.388889,0.413567,0.771246
SVM,0.875395,0.365412,0.433333,0.393174,0.752352
Naive Bayes,0.294560,0.096033,0.788889,0.171189,0.536592
Decision Tree,0.887719,0.381946,0.377778,0.389558,0.643071
K-Nearest Neighbors,0.883643,0.348016,0.266667,0.297903,0.628900
